In [1]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import add_dummy_feature
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
import joblib

Matplotlib is building the font cache; this may take a moment.


In [2]:
# Función para cargar pesos de los modelos
def load_models(models_dir):
    model_files = list(Path(models_dir).glob('*.pkl'))
    models = {}
    
    for file in model_files:
        try:
            model = joblib.load(file)
            models[file.stem] = model
        except Exception as e:
            print(f"Error cargando {file.name}: {str(e)}")
    
    return models

In [3]:
# Función para cargar el dataset de test
def load_test_data(data_dir):
    test_file = Path(data_dir) / 'Airbnb_Test.csv'
        
    if not test_file.exists():
        raise FileNotFoundError(f"El archivo {test_file} no se encontró")
        
    test_data = pd.read_csv(test_file)
        
    y_test = test_data['log_price']
    X_test = test_data.drop('log_price', axis=1)

    y_test = y_test.to_numpy()
    X_test = X_test.to_numpy()

    X_test = add_dummy_feature(X_test)
    
    return X_test, y_test

In [4]:

# Función para evaluar un modelo
def evaluate_model(weights, X_test, y_test):
    y_pred = X_test @ weights
    
    return {
        'MSE': mean_squared_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'MAE': mean_absolute_error(y_test, y_pred),
        'R2': r2_score(y_test, y_pred)
    }

In [5]:
models_dir = '../models'
data_dir = '../data/processed'
models = load_models(models_dir)

X_test, y_test = load_test_data(data_dir)

# Para el modelo polinomial
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_test_poly = poly_features.fit_transform(X_test)

results = {}
for name, model in models.items():
    # print(f"Evaluando modelo: {name}")
    # print(f"Modelo: {model}")
    try:
        if name == 'polynomial':
            y_pred = model @ X_test_poly.T
            results[name] = {
                'MSE': mean_squared_error(y_test, y_pred),
                'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
                'MAE': mean_absolute_error(y_test, y_pred),
                'R2': r2_score(y_test, y_pred)
            }
        else:
            results[name] = evaluate_model(model, X_test, y_test)
    except Exception as e:
        print(f"Error evaluando {name}: {str(e)}")

results_df = pd.DataFrame(results).T
results_df.sort_values(by='R2', ascending=False, inplace=True)
results_df

,MSE,RMSE,MAE,R2
bgd,0.187007,0.432443,0.316753,0.626104
ridge,0.187010,0.432447,0.316757,0.626097
svd,0.187011,0.432447,0.316759,0.626096
ecuacion_normal,0.187011,0.432447,0.316759,0.626096
sgd,0.187129,0.432584,0.316807,0.625860
lasso,0.268691,0.518354,0.384978,0.462786
sgd_sklearn,6.585197,2.566164,2.528658,-12.166256
bgd_sklearn,6.751044,2.598277,2.561443,-12.497847
polynomial,25.652879,5.064867,5.047272,-50.289639


### Conclusiones
Tras evaluar múltiples modelos lineales para la predicción de precios en Airbnb, el **Batch Gradient Descent (BGD)** con implementación propia emerge como una posible solución óptima. Con un R² de 0.626104, este modelo explica aproximadamente el 63% de la variabilidad en los precios, un resultado satisfactorio considerando la complejidad inherente a la determinación de precios en alojamientos temporales.

#### MSE (0.187007)
Este valor indica que el promedio del cuadrado de las diferencias entre los precios predichos y los reales es relativamente bajo. Dado que el MSE eleva al cuadrado los errores, este valor sugiere que el modelo no comete errores extremadamente grandes en sus predicciones, lo cual es crucial para mantener la confiabilidad en un sistema de recomendación de precios.

#### RMSE (0.432443)
Al ser la raíz cuadrada del MSE, este valor está en las mismas unidades que la variable objetivo, que en este caso es el logaritmo del precio. Dado que los precios están en escala logarítmica, un RMSE de 0.432443 significa que las predicciones tienen un factor de error multiplicativo de aproximadamente e^0.43 ≈ 1.54. En términos prácticos, esto implica que si el precio real es de $100, el modelo podría predecir entre $65 y $154 aproximadamente para el 68% de los casos (suponiendo distribución normal de errores). Esta magnitud de error es razonable considerando la variabilidad inherente en los precios de alojamientos temporales y proporciona un nivel de precisión adecuado para la toma de decisiones de fijación de precios.

#### MAE (0.316753)
Este valor representa la desviación absoluta promedio entre las predicciones y los valores reales. Es menor que el RMSE, lo que indica que hay algunos errores más grandes que influyen más en el RMSE. El MAE sugiere que, en promedio, las predicciones se desvían aproximadamente 0.32 unidades de los precios reales, lo que representa una precisión razonable para decisiones de fijación de precios.

La superioridad del BGD se evidencia en sus métricas de error consistentemente más bajas (MSE: 0.187007, RMSE: 0.432443, MAE: 0.316753) en comparación con los demás modelos. La cercanía en rendimiento entre BGD, Ridge, SVD y Ecuación Normal sugiere que se ha encontrado una solución robusta al problema, mientras que las implementaciones de sklearn para SGD/BGD y el modelo polinomial resultaron inadecuadas, evidenciado por sus valores R² negativos extremos. Esto probablemente se deba a que para este tipo de modelos es mejor realizar una optimización de hiperparametros, por ejemplo con GridSearchCV. Sin embargo se optó por no utilizar esto para "estar en igualdad de condiciones" con respecto a todos los modelos.